In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import re
import pycountry
import openpyxl
import random

random.seed(42)

ocde = [
    'Australia',
    'Austria',
    'Belgium',
    'Canada',
    'Chile',
    'Colombia',
    'Costa Rica',
    'Czechia',
    'Denmark',
    'Estonia',
    'Finland',
    'France',
    'Germany',
    'Greece',
    'Hungary',
    'Iceland',
    'Ireland',
    'Israel',
    'Italy',
    'Japan',
    'South Korea',
    'Latvia',
    'Lithuania',
    'Luxembourg',
    'Mexico',
    'Netherlands',
    'New Zealand',
    'Norway',
    'Poland',
    'Portugal',
    'Slovakia',
    'Slovenia',
    'Spain',
    'Sweden',
    'Switzerland',
    'Türkiye',
    'United Kingdom',
    'United States'
]

df = pd.read_json('../../data/ld/prokanon.ld_combs4.json')

df['count'] = df['count'].apply(lambda x: x if x >=300 else random.randint(1, 300))
df.sort_values(by='count',inplace=True)


df['cumsum'] = df['count'].cumsum()

suma = df['count'].sum()

df['cdf'] = df['cumsum'] / suma

df


,_id,age_range,seniority,country,gender,count,cumsum,cdf
5842,{'$oid': '65b0f45c17f3c67cf286ceaa'},"(25,34)",1,Kyrgyzstan,MALE,1,1,2.107659e-09
16085,{'$oid': '65b77b9e60aac84946a7936a'},"(35,54)",1,Zimbabwe,FEMALE,1,2,4.215317e-09
13267,{'$oid': '65b7766060aac84946a76a7c'},"(25,34)",9,Ethiopia,FEMALE,1,3,6.322976e-09
1765,{'$oid': '65b0eab517f3c67cf2869285'},"(35,54)",1,Gibraltar,FEMALE,1,4,8.430634e-09
419,{'$oid': '65b0e75a17f3c67cf2866f38'},"(25,34)",3,Saint Pierre and Miquelon,FEMALE,1,5,1.053829e-08
...,...,...,...,...,...,...,...,...
187,{'$oid': '65b0e6c717f3c67cf2866927'},"(25,34)",4,United States,FEMALE,7200000,431360152,9.091599e-01
181,{'$oid': '65b0e6c317f3c67cf2866900'},"(35,54)",3,United States,FEMALE,7600000,438960152,9.251781e-01
5298,{'$oid': '65b0f30417f3c67cf286c777'},"(25,34)",3,India,MALE,8500000,447460152,9.430932e-01
178,{'$oid': '65b0e6c117f3c67cf28668ea'},"(25,34)",3,United States,MALE,13000000,460460152,9.704928e-01


In [2]:
len(df[df['count'] == 0])/len(df)

0.0

In [3]:
df_aux = df['count'].value_counts().sort_index()[0:300]
df_aux = df_aux * df_aux.index
df_aux.sum()

np.int64(1034212)

In [4]:
df['count'].sum()

np.int64(474460152)

In [5]:
df_aux.sum()/df['count'].sum()

np.float64(0.0021797657730379854)

In [6]:
df_aux = df['count'].value_counts().sort_index()[0:1]
df_aux = df_aux * df_aux.index
df_aux.sum()

np.int64(22)

In [7]:
df_aux.sum()/df['count'].sum()

np.float64(4.636848828560844e-08)

In [8]:

res_geq090 = df[df['cdf'] >= 0.1]

idx_min = (res_geq090['cdf'] - 0.1).abs().idxmin()

results_90 = res_geq090.loc[idx_min]



res_geq095 = df[df['cdf'] >= 0.05]

idx_min = (res_geq095['cdf'] - 0.05).abs().idxmin()

results_95 = res_geq095.loc[idx_min]



res_geq099 = df[df['cdf'] >= 0.01]

idx_min = (res_geq099['cdf'] - 0.01).abs().idxmin()

results_99 = res_geq099.loc[idx_min]


In [9]:
results = pd.DataFrame(columns=['K^4_90', 'K^4_95', 'K^4_99', 'RK^4_90', 'RK^4_95', 'RK^4_99'])
results['K^4_90'] = results_90
results['K^4_95'] = results_95
results['K^4_99'] = results_99

In [10]:
results = results.T[['count']].T.reset_index(drop = True)
results['RK^4_90'] = (results['K^4_90'] / suma) * 100
results['RK^4_95'] = (results['K^4_95'] / suma) * 100
results['RK^4_99'] = (results['K^4_99'] / suma) * 100

In [11]:
results.to_parquet('../../data/analyzed/ld/LD_K^4_X_general.parquet')

In [12]:
countries_ocde_a2 = pd.DataFrame(pd.DataFrame(ocde,columns=['country'])['country'].apply(lambda x: pycountry.countries.lookup(x).alpha_2))

In [13]:
df['country'] = df['country'].str.strip()  
df['country'] = df['country'].replace({'Turkey': 'Türkiye'})

In [14]:
df_ocde = df[df['country'].apply(lambda x: pycountry.countries.lookup(x).alpha_2).isin(countries_ocde_a2['country'])].copy()
df_ocde.sort_values(by='count',inplace=True)

df_ocde['cumsum'] = df_ocde['count'].cumsum()

suma = df_ocde['count'].sum()

df_ocde['cdf'] = df_ocde['cumsum'] / suma

In [15]:

res_geq090 = df_ocde[df_ocde['cdf'] >= 0.1]

idx_min = (res_geq090['cdf'] - 0.1).abs().idxmin()

results_90 = res_geq090.loc[idx_min]



res_geq095 = df_ocde[df_ocde['cdf'] >= 0.05]

idx_min = (res_geq095['cdf'] - 0.05).abs().idxmin()

results_95 = res_geq095.loc[idx_min]



res_geq099 = df_ocde[df_ocde['cdf'] >= 0.01]

idx_min = (res_geq099['cdf'] - 0.01).abs().idxmin()

results_99 = res_geq099.loc[idx_min]

results = pd.DataFrame(columns=['K^4_90', 'K^4_95', 'K^4_99', 'RK^4_90', 'RK^4_95', 'RK^4_99'])
results['K^4_90'] = results_90
results['K^4_95'] = results_95
results['K^4_99'] = results_99

results = results.T[['count']].T.reset_index(drop = True)
results['RK^4_90'] = (results['K^4_90'] / suma) * 100
results['RK^4_95'] = (results['K^4_95'] / suma) * 100
results['RK^4_99'] = (results['K^4_99'] / suma) * 100

results.to_parquet('../../data/analyzed/ld/LD_K^4_X_general_ocde.parquet')
results

,K^4_90,K^4_95,K^4_99,RK^4_90,RK^4_95,RK^4_99
0,72000,31000,7000,0.024538,0.010565,0.002386


In [16]:
df_ocde[df_ocde['count']==0]['country'].drop_duplicates().count()

np.int64(0)

In [17]:
len(ocde)

38